# HOUM Challenge

In [57]:
from pathlib import Path

# Reemplazar con ruta del archivo (en caso de que notebook no se ejecute en el
# mismo directorio)
RESOURCE_DIR = Path(".")
PROPERTIES_DATA = RESOURCE_DIR / "properties.csv"
USERS_DATA = RESOURCE_DIR / "users.csv"
VISITS_DATA = RESOURCE_DIR / "visits.csv"

assert PROPERTIES_DATA.exists()
assert USERS_DATA.exists()
assert VISITS_DATA.exists()

## Supuestos

Para simplificar el desarrollo de la tarea se tomarán los siguientes supuestos:

* Para el periodo de análisis de interés, se asumirá que los registros (`*.csv`) pueden cargarse en memoria (en caso contrario no cambia mucho el algoritmo, pero hay que entrar a usar disco y cargas temporales de datos)
* Como hay suficiente RAM para cargar los datos, se utilizará pandas. En la vida real, uno tendría acceso a un data warehouse, donde se podrían computar los valores de interés.

In [58]:
import pandas as pd


visits = pd.read_csv(VISITS_DATA)
visits.head()

,scheduled_id,property_id,begin_date,end_date,type_visit,status
0,169548,1,2022-01-13T10:00:00-03:00,2022-01-13T12:00:00-03:00,Visit,Cancelled
1,184763,1,2022-01-26T18:00:00-03:00,2022-01-26T20:00:00-03:00,Visit,Cancelled
2,186092,1,2022-01-28T12:00:00-03:00,2022-01-28T14:00:00-03:00,Visit,Cancelled
3,182497,1,2022-01-23T11:00:00-03:00,2022-01-23T13:00:00-03:00,Visit,Cancelled
4,182396,1,2022-01-23T10:00:00-03:00,2022-01-23T12:00:00-03:00,Visit,Cancelled


In [59]:
visits.type_visit.value_counts()

Visit    425
Name: type_visit, dtype: int64

In [60]:
visits.status.value_counts()

Cancelled    286
Done         139
Name: status, dtype: int64

## Cantidad de visitas

De la tabla de visitas, se obtiene que la cantidad de visitas ocurridas en el periodo de interés fueron 139 (ya que 286 fueron canceladas)

In [61]:
users = pd.read_csv(USERS_DATA)
users.head()

,property_id,user_id,first_name,last_name,address
0,31,1,Josephine,Darakjy,4 B Blue Ridge Blvd
1,34,2,Art,Venere,8 W Cerritos Ave #54
2,48,3,Lenna,Paprocki,639 Main St
3,27,4,Donette,Foller,34 Center St
4,40,5,Simona,Morasca,3 Mcauley Dr


In [62]:
users.groupby(["user_id"]).size().mean()

1.0

## Promedio de propiedades por usuario

Todos los usuarios de la base de datos entregada tienen una sola propiedad, por lo tanto el promedio de propiedades por usuario es 1.

In [63]:
properties = pd.read_csv(PROPERTIES_DATA)
properties.head()

,property_id,type_house,business_type,bedrooms,bathrooms,parking_lots,services,balcony,pool,latitude,longitude,localidad,city,region,country
0,1,departamento,Rental,1,1,1,3.0,0,False,4.870956,-74.058040,Chí­A,Cundinamarca,Región De Cundinamarca,Colombia
1,2,departamento,Rental & Sale,1,1,2,NaN,1,False,4.623068,-74.074030,Teusaquillo,Bogotá,Región De Cundinamarca,Colombia
2,3,departamento,Rental,2,2,2,NaN,0,False,4.723909,-74.042336,Usaquen,Bogotá,Región De Cundinamarca,Colombia
3,4,departamento,Rental & Sale,3,2,1,0.0,0,False,4.721716,-74.057976,Suba,Bogotá,Región De Cundinamarca,Colombia
4,5,departamento,Rental & Sale,2,3,2,0.0,1,False,4.688163,-74.043300,Usaquen,Bogotá,Región De Cundinamarca,Colombia


# Temperatura promedio de visitas realizadas por usuario con `ID = 2`

Con las fuentes de datos entregadas, no queda claro qué es lo que se pregunta. La única forma de relacionar usuarios con visitas es utilizando el ID de la propiedad, y eso asumiendo que todas las relaciones en la tabla `users` representan que el usuario de id `user_id` es dueño de la propiedad `property_id`.

Si lo que se pregunta es diferente (que pareciera ser así), el problema está mal formulado y habría que indagar en la fuente de datos para conseguir los datos correctos.

Por otro lado, tendría más sentido preguntar sobre las visitas a la propiedad con `id = 2` en base al enunciado al problema. Por lo tanto, como supuesto se toma:

*Interesa responder la temperatura promedio de visitas realizadas a la propiedad con `ID = 2`*

Habría que esperar que Carlos aclarara los requerimientos para poder resolver el problema de la forma esperada.

De cualquier forma, en términos técnicos no cambia la solución (habría que adaptarla a los datos correctos una vez se obtengan).

In [85]:
# Inner Join con datos
visits_property_data = pd.merge(visits, properties, on="property_id")
visits_property_data.head()

,scheduled_id,property_id,begin_date,end_date,type_visit,status,begin_date_datetime,end_date_datetime,day_diff,type_house,...,parking_lots,services,balcony,pool,latitude,longitude,localidad,city,region,country
0,169548,1,2022-01-13T10:00:00-03:00,2022-01-13T12:00:00-03:00,Visit,Cancelled,2022-01-13 10:00:00-03:00,2022-01-13 12:00:00-03:00,0,departamento,...,1,3.0,0,False,4.870956,-74.05804,Chí­A,Cundinamarca,Región De Cundinamarca,Colombia
1,184763,1,2022-01-26T18:00:00-03:00,2022-01-26T20:00:00-03:00,Visit,Cancelled,2022-01-26 18:00:00-03:00,2022-01-26 20:00:00-03:00,0,departamento,...,1,3.0,0,False,4.870956,-74.05804,Chí­A,Cundinamarca,Región De Cundinamarca,Colombia
2,186092,1,2022-01-28T12:00:00-03:00,2022-01-28T14:00:00-03:00,Visit,Cancelled,2022-01-28 12:00:00-03:00,2022-01-28 14:00:00-03:00,0,departamento,...,1,3.0,0,False,4.870956,-74.05804,Chí­A,Cundinamarca,Región De Cundinamarca,Colombia
3,182497,1,2022-01-23T11:00:00-03:00,2022-01-23T13:00:00-03:00,Visit,Cancelled,2022-01-23 11:00:00-03:00,2022-01-23 13:00:00-03:00,0,departamento,...,1,3.0,0,False,4.870956,-74.05804,Chí­A,Cundinamarca,Región De Cundinamarca,Colombia
4,182396,1,2022-01-23T10:00:00-03:00,2022-01-23T12:00:00-03:00,Visit,Cancelled,2022-01-23 10:00:00-03:00,2022-01-23 12:00:00-03:00,0,departamento,...,1,3.0,0,False,4.870956,-74.05804,Chí­A,Cundinamarca,Región De Cundinamarca,Colombia


In [151]:
from abc import ABC, abstractmethod
import requests

API_KEY = "<INSERTAR SU API KEY>"

class VisualCrossingServicesBase:
    __base_uri = (
        "https://weather.visualcrossing.com/"
        "VisualCrossingWebServices/rest/services")
    
    key = ""

    @property
    def base_uri(self):
        return self.__base_uri

    @property
    def uri(self):
        return f"{self.base_uri}/{self.key}"


class VisualCrossingServicesTimeline(VisualCrossingServicesBase):
    key = "timeline"
    def __init__(self, api_key):
        self.api_key = api_key

    def get(self, location, date1, date2):
        uri = f"{self.uri}/{location}/{date1}/{date2}"
        res = requests.get(uri, params={
            "key": self.api_key,
            "elements": "datetime,temp,conditions",
            "include": "days"
        })
        res.raise_for_status()
        return res.json()


In [152]:
# Validar que todas las visitas ocurran el mismo día
visits["begin_date_datetime"] = pd.to_datetime(visits["begin_date"])
visits["end_date_datetime"] = pd.to_datetime(visits["end_date"])
visits["day_diff"] = visits.apply(lambda row: (row["end_date_datetime"] - row["begin_date_datetime"]).days, axis=1)
visits["day_diff"].describe()

count    425.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
Name: day_diff, dtype: float64

In [156]:
def get_temperature(row):
    timeline = VisualCrossingServicesTimeline(API_KEY)
    latitude = row["latitude"]
    longitude = row["longitude"]
    location = f"{latitude},{longitude}"
    date1 = row["begin_date"].split("T")[0]
    date2 = row["end_date"].split("T")[0]
    api_data = timeline.get(location, date1, date2)
    print(api_data)
    result = pd.Series([api_data["days"][0]["temp"], api_data["days"][0]["conditions"]])
    return result
    

test_row = visits_property_data.loc[0, :]
get_temperature(test_row)

{'queryCost': 1, 'latitude': 4.870956, 'longitude': -74.05804, 'resolvedAddress': '4.870956,-74.05804', 'address': '4.870956,-74.05804', 'timezone': 'America/Bogota', 'tzoffset': -5.0, 'days': [{'datetime': '2022-01-13', 'temp': 55.1, 'conditions': 'Rain, Partially cloudy'}]}


0                      55.1
1    Rain, Partially cloudy
dtype: object

In [157]:
# Supuesto, utilizar property_id en lugar de user_id
PROPERTY_ID = 2
q1 = visits_property_data[
                          (visits_property_data["property_id"] == PROPERTY_ID)
                          & (visits_property_data["status"] == "Done")]
q1[["temp", "conditions"]] = q1.apply(get_temperature, axis=1)

{'queryCost': 1, 'latitude': 4.6230683, 'longitude': -74.07403, 'resolvedAddress': '4.6230683,-74.07403', 'address': '4.6230683,-74.07403', 'timezone': 'America/Bogota', 'tzoffset': -5.0, 'days': [{'datetime': '2022-01-20', 'temp': 58.2, 'conditions': 'Rain, Partially cloudy'}]}


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [158]:
q1[["temp", "conditions"]]

,temp,conditions
9,58.2,"Rain, Partially cloudy"


## Query 1: Temperatura 

La temperatura promedio para `property_id = 2` fue de 58.2 ºF

In [118]:
visits_property_data.shape

(425, 23)

In [125]:
visits_property_data.info()

array(['Cundinamarca', 'Bogotá'], dtype=object)

In [126]:
properties.country.unique()

array(['Colombia'], dtype=object)

## Optimización

Para reducir el costo de la query a la API, consideramos la cantidad de ciudades y países en el conjunto de datos, y como las visitas duran un día, tomamos el supuesto en que las condiciones climáticas no cambiarán demasiado dentro de la visita (en algunos casos podría ser, pero la intuición dice que no es tan probable). Entonces reduciremos de 425 consultas a sólo 2.

In [172]:
mask = visits_property_data.city == "Cundinamarca"
min_date = visits_property_data[mask].begin_date_datetime.min().strftime("%Y-%m-%d")
max_date = visits_property_data[mask].begin_date_datetime.max().strftime("%Y-%m-%d")

query = ("Cundinamarca,Colombia", min_date, max_date)
print(min_date, "/", max_date)

2022-01-03 / 2022-02-28


In [173]:
timeline = VisualCrossingServicesTimeline(API_KEY)
res1 = timeline.get(*query)

In [174]:
index = {}
address = res1["address"]
index[address] = {}
for day in res1["days"]:
    index[address][day["datetime"]] = {
        "conditions": day["conditions"],
        "temp": day["temp"]
    }

In [175]:
mask = visits_property_data.city == "Bogotá"
min_date = visits_property_data[mask].begin_date_datetime.min().strftime("%Y-%m-%d")
max_date = visits_property_data[mask].begin_date_datetime.max().strftime("%Y-%m-%d")

query = ("Bogotá,Colombia", min_date, max_date)
print(min_date, "/", max_date)

2022-01-01 / 2022-02-28


In [176]:
res2 = timeline.get(*query)

In [177]:
address = res2["address"]
index[address] = {}
for day in res2["days"]:
    index[address][day["datetime"]] = {
        "conditions": day["conditions"],
        "temp": day["temp"]
    }

In [179]:
def get_weather_features(row):
    key = f"{row.city},{row.country}"
    date = row.begin_date_datetime.strftime("%Y-%m-%d")

    result = index[key][date]
    return pd.Series([result["temp"], result["conditions"]])

In [181]:
visits_property_data[["temp", "conditions"]] = visits_property_data.apply(
    get_weather_features, axis=1)

visits_property_data.head()

,scheduled_id,property_id,begin_date,end_date,type_visit,status,begin_date_datetime,end_date_datetime,day_diff,type_house,...,balcony,pool,latitude,longitude,localidad,city,region,country,temp,conditions
0,169548,1,2022-01-13T10:00:00-03:00,2022-01-13T12:00:00-03:00,Visit,Cancelled,2022-01-13 10:00:00-03:00,2022-01-13 12:00:00-03:00,0,departamento,...,0,False,4.870956,-74.05804,Chí­A,Cundinamarca,Región De Cundinamarca,Colombia,55.1,"Rain, Partially cloudy"
1,184763,1,2022-01-26T18:00:00-03:00,2022-01-26T20:00:00-03:00,Visit,Cancelled,2022-01-26 18:00:00-03:00,2022-01-26 20:00:00-03:00,0,departamento,...,0,False,4.870956,-74.05804,Chí­A,Cundinamarca,Región De Cundinamarca,Colombia,58.9,"Rain, Partially cloudy"
2,186092,1,2022-01-28T12:00:00-03:00,2022-01-28T14:00:00-03:00,Visit,Cancelled,2022-01-28 12:00:00-03:00,2022-01-28 14:00:00-03:00,0,departamento,...,0,False,4.870956,-74.05804,Chí­A,Cundinamarca,Región De Cundinamarca,Colombia,58.1,"Rain, Partially cloudy"
3,182497,1,2022-01-23T11:00:00-03:00,2022-01-23T13:00:00-03:00,Visit,Cancelled,2022-01-23 11:00:00-03:00,2022-01-23 13:00:00-03:00,0,departamento,...,0,False,4.870956,-74.05804,Chí­A,Cundinamarca,Región De Cundinamarca,Colombia,59.0,"Rain, Partially cloudy"
4,182396,1,2022-01-23T10:00:00-03:00,2022-01-23T12:00:00-03:00,Visit,Cancelled,2022-01-23 10:00:00-03:00,2022-01-23 12:00:00-03:00,0,departamento,...,0,False,4.870956,-74.05804,Chí­A,Cundinamarca,Región De Cundinamarca,Colombia,59.0,"Rain, Partially cloudy"


## Temperatura promedio para los días con lluvia

Se asume que si llovió, independiente del tipo de lluvia, se considerará lluvia. Por otro lado, sólo se consideran visitas que no fueron canceladas

In [187]:
mask = (visits_property_data.conditions.str.match("rain", case=False)
        & (visits_property_data.status == "Done"))

visits_property_data[mask].temp.mean()

57.76803278688521

La temperatura promedio en los días de lluvia fue 57.8 ºF

## Temperatura promedio para visitas en Suba

En los datos no está el campo `locality`, por lo que habría que contactar a Carlos para saber si está correcto lo que dice. Por otro lado, se puede estimar haciendo una query a la API, considerando las mismas fechas que alguna de las consultas anteriores. Como suba queda en Bogotá un estimado grueso sería la temperatura media en bogotá.

In [192]:
mask = ((visits_property_data.city == "Bogotá")
        & (visits_property_data.status == "Done"))
visits_property_data[mask].temp.mean()

57.733050847457605

Se obtiene que la temperatura media estimada para la localidad de Suba, sería de 57.7 ºF.